# Prediction Module

Contained in this notebook are the steps needed to train the neural network models implemented for Protein Structural Features prediction. This one foollows the same rules of the InputPrep Module, but no need for external libs. 27/08/2020

#### Installation of Libraries

**The container version and name is:**
- TF2.1.0
- tensorflow/tensorflow:latest-gpu-py3-jupyter

##### Image ran: tensorflow/tensorflow:1.15.0-gpu-py3-jupyter
docker pull tensorflow/tensorflow:1.15.0-gpu-py3-jupyter

To run the container, one could also make an alias, as so:

alias docker_tf='docker run -v /LOCAL/VOLUME/:/tf/CONTAINER_VOLUME -p 8888:8888 --rm --runtime=nvidia -it --rm tensorflow/tensorflow:latest-gpu-py3-jupyter'

### How to Run this notebook:
1. Prepare the files as done by input prep module or open any dump of the data;
2. Make sure you have the following packages installed:
>> - Python 3 <br>
>> - Tensorflow <br>
>> - Scikit Learn <br>
>> - Matplotlib <br>
>> (you can download docker and pull/run the above mentioned container) <br>
3. Execute the cells in sequence.
>> 1. Observe cell description before running. Some of them load files produced on previous steps. So you can continue to explore stuff and skip some cells.


### Considerações:
NEssa sessão, só há observações sobre o modelo e que podem mudar, assim como notas gerais e etc.

<br>1- A rede, do jeito que tá, está funcionando para predição de Theta. mas:
<br>1.1- Preciso treinar com uma loss que não considere os ângulos dos loci 0 e 1.
<br>2- Preciso de uma loss com pesos também. Ela regride, e calcula o erro dado um peso para uma range. É possível, viável e útil fazer isso?
<br>3- Implementar o restart do estado do Adam em caso de stalling de val_loss.
<br>3.1- Não façço a mínima ideia de como fazer isso, mas vou ter, talvez, que sobrecarregar o CallBack early stop do keras. Fazer um do zero talvez não seja o melhor approach. Posso ver se dá pra mudar alguma função tipo on_epoch_end
<br>4- E se eu colocar uma outra "mini rede pra predizer a range da distribuição" do alfabeto? Vai ser viagem
<br>5- Preciso refatorar o gerador de dados pra aplicar o ang_type (theta, phi ou PhiPsi)

In [ ]:
# Padrão ouro de parâmetros (por enquanto)
'''
Batch Size = 32|53
Datasize = 64|53
LR0 = 0.001
BN after Merge = True
RLROP = False
opt = Adam
Epochs = 140
Loss1 = ANG_MAE
Loss2 = mae
loss3 = RMSE
'''
# Rede
'''
config = {'model':'resBiLSTM',
            'input_shape': [256,46],
         'rnn_width': 250,
         'rnn_depth':2,
         'rnn_dropout': 0.2,
         'ang_mode': 'alphabet',
         'alphabet_parms': {'n_alphas': 10, 'n_angles': 1, 'alpha_range': [0.,2*np.pi], 'force_alpha_range': False}}

#[0.5,2.7]
'''

# Melhor padrão foi o Nadam 0003 RLROP, que na vdd usou lr inicial de 0.001 normal. Ou seja, tem que fazer triplicata.
# Vamos ver agora com o banco de dados todo. VOu mandar pra JAG e vou treinar lá, shablau.
# Agora eu vou programar pra diedrais. Agora o Bicho pega, pq a range é maior, e efetivamente a distribuição é radial.



In [ ]:
# procurar por quais descritores usei!! urgente!! mudar os descritores depois

### Global and Control Variables
<br>
The variables listed and declared here will be used to control the entire process. Each of these will be described using comments following the declaration. The parameters written here **are the defaults**. Don't change unless you know exactly what you are doing. For instance, changin _p_number_ could generate a buffer overflow on your computer, and you will end up being mad.

In [1]:
import subprocess
import sys
import os
from os import listdir
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow.keras as keras

import tensorflow as tf
from generators4 import AngleDataGenerator
from Utils import Utils as utils
#SUpresses TF warnings
import numpy as np
import model3
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

### List all training files

In [2]:
train_dir = 'train_dih/'
train_files = listdir(train_dir)
val_dir = 'valid_dih/'
val_files = listdir(val_dir)
#test_dir = 'testing_data/'
#test_files = listdir(test_dir)

### Map all filenames to a given y

In [3]:
# In this case, we are using len(all_files)//4 because we have 4 different data types inside the folder
# gotta fix this ASAP
seq_dih_map_train = { 'x_{:04d}.npy'.format(i+1):'y_{:04d}.npy'.format(i+1) for i in np.arange(len(train_files)//2)}
seq_dih_map_val = { 'x_{:04d}.npy'.format(i+1):'y_{:04d}.npy'.format(i+1) for i in np.arange(71)}

In [4]:
train_ids = dict(list(seq_dih_map_train.items()))
valid_ids = dict(list(seq_dih_map_val.items()))

In [7]:
# Data Generators specification
train_gen = AngleDataGenerator(train_ids,16,data_dir='train_dih/',ang_type='dih')
valid_gen = AngleDataGenerator(valid_ids,53,data_dir='valid_dih/',ang_type='dih')


### Prepares and compiles the model

#### Model Configuration

In [ ]:
config = {'model':'resBiLSTM',
            'input_shape': [256,46],
         'rnn_width': 250,
         'rnn_depth':2,
         'rnn_dropout': 0.2,
         'ang_mode': 'alphabet',
         'alphabet_parms': {'n_alphas': 10, 'n_angles': 1, 'alpha_range': [0.,2*np.pi], 'force_alpha_range': False}}

#[0.5,2.7]

#### Model Declaration

In [ ]:
type(config)

In [ ]:


mod_lib = model3.AnglePredictor(config)
model = mod_lib.build_model()

### Test Alternative Models

In [8]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Bidirectional, BatchNormalization, Masking, Lambda, LSTM, Dense, RepeatVector, TimeDistributed,Conv1D, Input, add
from layer_utils_3 import Alphabet
import os
from random import randint
import numpy as np
import tensorflow as tf

In [ ]:
from tcn import TCN, tcn_full_summary

In [ ]:
x_1 = np.load('train_70_dih/x_0001.npy')

In [9]:

import model_teste3 as MODEL

In [10]:
model_ = MODEL.AnglePredictor()

In [ ]:
inputs = tf.keras.Input(shape=(256,46))
outputs = model_(inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 46)]         0         
_________________________________________________________________
angle_predictor (AnglePredic (None, 256, 2)            147646    
Total params: 147,646
Trainable params: 143,550
Non-trainable params: 4,096
_________________________________________________________________


In [ ]:
a = model.predict(x_1)

In [ ]:
count_zero=0
for r in a[0]:
    if np.all(r == 0):
        count_zero+=1
print(count_zero)

**Testing TCN Layer**

In [ ]:
class Angularization():
    '''
    Angularization Class
    In this class are contained the means to process the core model
    outputs into angles.
    '''
    @staticmethod
    def regression_tanh(inputs,n_angles):
        x = inputs
        x = TimeDistributed(Dense(n_angles,activation='tanh'))(x)
        x = Lambda(lambda x: x * 3.141592653589793,name='lambda_tanh')(x)
        return x

    @staticmethod
    def alphabet(inputs,n_angles, n_alphas, alpha_range):
        x = inputs
        x = Alphabet(n_alphas, n_angles)(x)
        return x

In [ ]:
class CoreModels():
    @staticmethod
    def ResLSTM(inputs, rnn_width=180, rnn_depth=4, rnn_dropout=0.8, bn=True):
        x = inputs
        for i in range(rnn_depth):
            #i < rnn_depth - 1
            x_rnn = Bidirectional(LSTM(rnn_width, recurrent_dropout=rnn_dropout,
                                       dropout=rnn_dropout, return_sequences=True))(x)
            if i > 0:
                x = add([x, x_rnn])
                x = BatchNormalization()(x) if bn else x
            else:
                x = x_rnn
        return x

    @staticmethod
    def BiLSTM(inputs):
        x=inputs
        rnn_width=100 
        rnn_depth=2
        rnn_dropout=0.2
        for cell_count in range(rnn_depth):
            x = Bidirectional(LSTM(rnn_width, recurrent_dropout=rnn_dropout, dropout=rnn_dropout, return_sequences=True))(x)
        return x
    
    @staticmethod
    def StackedTCN(inputs, stacks=[64,32], nb_stacks=2, kernel_sizes=[2,2]):
        '''
        Stacked Temporal Convolutional Network
        COnsists of stacks of TCNs, as implemented by Philipp Remy at
        https://github.com/philipperemy/keras-tcn. Parameters are:
        stacks: (list) are the numbers of filters
        nb_stacks: (int) the number of stacks of residual blocks on the TCN layer (lib stuff)
        kernel_sizes: (list) list of kernel sizes for the TCN stacks.
        
        '''
        x = inputs
        for stack in stacks:
            x = TCN(nb_filters=stack, kernel_size=2, nb_stacks=2, dilations=[1, 2, 4, 8, 16],
            padding='causal', use_skip_connections=True, dropout_rate=0.2, return_sequences=True,
            activation='elu', kernel_initializer='he_normal', use_batch_norm=True)(x)
        return x
    

In [ ]:
i = Input((250,46))

In [ ]:
o = CoreModels.StackedTCN(i)

In [ ]:
o = Angularization.alphabet(o,2,10,[-np.pi,np.pi])

In [ ]:
m = Model(inputs=i,outputs=o)

In [ ]:
a = m.predict(x_1)

In [ ]:
# alphabet class almost done
# What has to be done (by order of priority)
# 1- Masking propagation (support?) through TCN layer
# 2- Training loop
# 3- Angle Vectorization support for Alphabet Layer
# 4- Geometric Layer

class Alphabet(tf.keras.layers.Layer):
    def __init__(self,n_alphas=10, n_angles=2, alpha_range = [0,1],
                 force_alpha_range=True, name='alphabet_layer', **kwargs):
        
        '''
        Alphabet Layer:
        This layer consists of a Densely connected network followed by a linear/angular regression operation.
        The main purpose is to convert the information gathered from the core models into a probability distribution
        via softmax function. By doing this we then perform the "angular regression" and achieve angle prediction.
        
        Angular prediction comes in two flavors: angular regression (as of now) and vectorized (later on).
        Layer parameters:
        n_alphas: (int) number of angle alphabet units to consider
        n_angles: (int) number of angles to predict (2: [phi, psi], 3: [phi, psi, omega])
        alpha_range: (list,tuple) the initialization range for the alphabet.
        force_alpha_range: (bool) wether or not to force the alphabet to stay on the initial range.
        
        Just a disclaimer:
        It doest support """" masking """" to some extent. It means that the layer receives a 'mask' tensor
        propagated from previous layers (originally from Masking) and casts it to float32. Followed by a 
        tensor product between float_mask and output from Alphabet. The idea is that, by passing the mask and
        performing the product, BP algorithm will gradually propagatethe importance of sequence length.
        I have no clues whatsoever if this will work as intended, but  it is a try. 
        '''
        
        super(Alphabet, self).__init__(**kwargs)
        self.n_angles = n_angles
        self.n_alphas = n_alphas
        #self.alphabet = alpabet
        self.alpha_range=alpha_range
        #assert type(self.alpha_range) in [list,tuple], "alpha_range must be a tuple or list of values."
        
        self.force_alpha_range = force_alpha_range
        
    def init_alphabet(self):
        '''
        Initializes the Angles Alphabet Matrix
        returns:
            Alphabet: matrix of floats of shape (n_alphas, n_angles)
        '''
        return tf.random.uniform(shape=(self.n_alphas, self.n_angles),
                                     minval=self.alpha_range[0], maxval=self.alpha_range[1],
                                     dtype='float32')
    def compute_mask(self, inputs, mask):
        # Time step masks must be the same for each input.
        # This is because the mask for an RNN is of size [batch, time_steps, 1],
        # and specifies which time steps should be skipped, and a time step
        # must be skipped for all inputs.
        # TODO(scottzhu): Should we accept multiple different masks?
        '''
        mask = nest.flatten(mask)[0]
        output_mask = mask if self.return_sequences else None
        if self.return_state:
            state_mask = [None for _ in self.states]
            return [output_mask] + state_mask
        else:
            return output_mask
        '''
        if mask is not None:
            return mask
    
    def get_config(self):
        # overiding get_config function
        # to enable layer serialization

        config = super().get_config().copy()
        config.update({
                    'n_alphas': self.n_alphas, 
                    'n_angles': self.n_angles,
                    'alpha_range': self.alpha_range,
                    'force_alpha_range': self.force_alpha_range
                    })
        
        return config
    
    def build(self, input_shape):
        self.dense_unit = TimeDistributed(tf.keras.layers.Dense(self.n_alphas,
                                                activation='relu',
                                                use_bias=True,
                                                kernel_initializer="glorot_uniform",
                                                bias_initializer="zeros"))
        
        self.alphabet = tf.Variable(self.init_alphabet(), trainable = True,name='alphabet')
        
    def call(self, input, mask=None):
        '''
        This method is called everytime the layer is invoked
        by the model. Actually what it does is passing the input by a 
        TimeDistributed Dense and then softmax it along the last axis and
        perform the weighted average ("angular regression") over the
        alphabet array. Which, in turn, is also a learnable parameter.4
        '''
        outputs = self.dense_unit(input)
        
        outputs = tf.keras.activations.softmax(outputs)
        outputs = tf.einsum('ij,kbi->kbj', self.alphabet, outputs)
        if mask is not None:
            mask = tf.cast(mask, "float32")
            outputs = outputs * mask[..., tf.newaxis]
        return outputs
        

In [ ]:
batch_size, timesteps, input_dim = 1, 256, 46
n_alphas, n_angles = 10, 2
i = Input((timesteps, input_dim))
mask = Masking(0)(i)

In [ ]:
# Core Learning Layer
# Gotta change the Sequence Training Data Generator later
# Change TCN layer to compute a mask. Actually need to change both the TCN and the Residual Block
o = TCN(nb_filters=64, kernel_size=2, nb_stacks=2, dilations=[1, 2, 4, 8, 16],
        padding='causal', use_skip_connections=True, dropout_rate=0.2, return_sequences=True,
        activation='elu', kernel_initializer='he_normal', use_batch_norm=True)(mask)  # The TCN layers are here.
o = TCN(nb_filters=32, kernel_size=2, nb_stacks=2, dilations=[1, 2, 4, 8],
        padding='causal', use_skip_connections=False, dropout_rate=0.1, return_sequences=True,
        activation='elu', kernel_initializer='he_normal', use_batch_norm=True)(o)  # The TCN layers are here.
############################################################################
# Needs a masking multiplication layer just over here                      
# The masking multiplication layer consists of an einsum layer, just that
# It will multiply the TCN results by the binary mask received on input.
############################################################################

#o = LSTM(64,return_sequences=True)(mask)

# Logit construction layer
o = Alphabet(10,2)(o)

#o = TimeDistributed(Dense(n_clusters))(o)
#o = Tester(2)(o)
m = Model(inputs=[i], outputs=[o])
m.compile(optimizer='adam', loss='mse')


#### Layer Alphabet
O novo layer Alphabet será definido a partir da implementação de m3h0w as instruções do layer são como se segue:<br>
0- No início do treinamento: inicia os clusters na range $[l,u]$. <br>
1- Recebe um tensor do modelo core (TCN, LSTM, etc) na forma (batch, Seq_Len, Features) <br>
2- Passa esse tensor por um dense + softmax pra pegar **logits** com shape $(batch, seqLen, logits)$ <br>
3- Clipa em clusters em $[-\pi,\pi]$. <br>
4- Einsum de **logits** com clusters pra pegar **angles** <br>

In [ ]:
# Need a masking layer to work properly
# 

class Alphabet(tf.keras.layers.Layer):
    def __init__(self,n_alphas=20, n_angles=2, alpha_range = [-np.pi,np.pi],
                 force_alpha_range=True, name='alphabet_layer', **kwargs):
        #super(Alphabet, self).__init__()
        
        self.n_alphas = n_alphas
        self.n_angles = n_angles
        self.alpha_range = alpha_range
        # the values for the dense unit will be changed later on
        
        self.force_alpha_range=force_alpha_range
        super(Alphabet).__init__(**kwargs)
    def init_alphabet(self):
        '''
        Initializes the Angles Alphabet Tensor
        returns:
            Alphabet: tensor of floats of shape (n_alphas, n_angles)
        '''
        return tf.random.uniform(shape=(self.n_alphas, self.n_angles),
                                     minval=self.alpha_range[0], maxval = self.alpha_range[1],
                                     dtype='float32')
    def build(self, input_shape):
        print(input_shape)
        self.alphabet = tf.Variable(self.init_alphabet(), trainable = True,name='alphabet')
        self.dense_unit = tf.keras.layers.Dense(n_alphas,
                                                activation='elu',
                                                use_bias=True,
                                                kernel_initializer="glorot_uniform",
                                                bias_initializer="zeros")

In [ ]:
class MinimalRNNCell(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        self.units = units
        self.state_size = units
        #self.state_size = [tf.TensorShape([units])]
        super(MinimalRNNCell, self).__init__(**kwargs)
    def build(self, input_shape):
        print(type(input_shape))
        self.kernel = self.add_weight(shape=(input_shape[0][-1], self.units),
                                      initializer='uniform',
                                      name='kernel')
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units),
            initializer='uniform',
            name='recurrent_kernel')
        self.built = True
    def call(self, inputs, states):
        prev_output = states[0]
        h = K.dot(inputs[0], self.kernel)
        output = h + K.dot(prev_output, self.recurrent_kernel)
        return output, [output]   

# Let's use this cell in a RNN layer:
cell = MinimalRNNCell(32)
input_1 = tf.keras.Input((None, 5))
input_2 = tf.keras.Input((None, 5))
layer = RNN(cell)
y = layer((input_1, input_2))

In [ ]:
a = tf.random.uniform(shape=(10, 2),
                                     minval=-np.pi, maxval = np.pi,
                                     dtype='float32')

In [ ]:
a.shape

In [ ]:
tf.multiply

In [ ]:
m.summary()

In [ ]:
pred = m.predict(x_1)

In [ ]:
count_zero=0
for r in pred[0]:
    if np.all(r == 0):
        count_zero+=1
print(count_zero)

### Preparing training data

In [ ]:
test_dir = 'testing_data/'
test_files = listdir(test_dir)
seq_dih_map_test = { 'x_{:04d}.npy'.format(i+1):'y_{:04d}.npy'.format(i+1) for i in np.arange(len(test_files)//4)}

## Loss Function and Callbacks

In [12]:
# This function is being tested as of now. But will become part of the workflow asap
def angular_MAE(y_true, y_pred):
    return tf.reduce_mean(tf.abs(tf.atan2(tf.sin(y_true - y_pred), tf.cos(y_true - y_pred))), axis=-1)

def rmse(y_true, y_pred):
    return tf.math.sqrt(tf.math.reduce_mean(tf.math.square(y_pred[:,3:,:] - y_true[:,3:,:])))

In [17]:

class SaveTestPredCallback(tf.keras.callbacks.Callback):
    ' Runs prediction on test set and stores the results inside the test_res folder'
    def __init__(self, x_to_pred=None):
        self.x_p=x_to_pred
    
    
    def on_epoch_end(self,epoch,logs=None):
        
        epoch_pred = self.model.predict_on_batch(self.x_p)
        np.save('testing_data/teste_pred_{}.npy'.format(epoch),epoch_pred)
        del epoch_pred


RLROP = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, verbose=1, mode='min',
    min_delta=0.1, cooldown=10, min_lr=0.000000011)
        
import resource
class MemoryCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, log={}):
        print('\n{}'.format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss*(1/1024*1024)))

class FollowOPT(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, log={}):
        opt_parms = [self.model.optimizer.lr.numpy(), self.model.optimizer.iterations.numpy(), self.model.optimizer.decay.numpy()]
        print('\nLR: {} | Iter: {} | Decay: {}\n'.format(*opt_parms))
#class RestartOpt(tf.keras)
# keep this commented for now
# 
# tensorboard callback
#from time import datetime
#logs = "logs/lr_001_RLROP_ang_mae_batchNorm_bottleneck" #+ datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/TCN_run1_alphabet_dih_07092020"
fo = FollowOPT()
tbcb = tf.keras.callbacks.TensorBoard(log_dir = log_dir,
                                                 histogram_freq = 1,
                                                write_images=False)#,
                                                 #profile_batch = '500,520')
        
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
#opt = tf.keras.optimizers.SGD(learning_rate=0.0001)
#mc = SaveTestPredCallback(x_test)

### Model Training

In [15]:
model.compile(loss=angular_MAE, optimizer=opt, metrics=['mae',rmse,'mse'])
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 46)]         0         
_________________________________________________________________
angle_predictor (AnglePredic (None, 256, 2)            147646    
Total params: 147,646
Trainable params: 143,550
Non-trainable params: 4,096
_________________________________________________________________


In [16]:

history = model.fit(x=train_gen,
      epochs=100, callbacks=[tbcb, fo],#, RLROP],
      validation_data=valid_gen,   
      use_multiprocessing=True,
     workers = 6)

Epoch 1/140
394/394 [==============================] - ETA: 0s - loss: 0.5603 - mae: 0.6714 - rmse: 1.3124 - mse: 1.7666
LR: 9.999999747378752e-05 | Iter: 394 | Decay: 0.0

394/394 [==============================] - 73s 185ms/step - loss: 0.5603 - mae: 0.6714 - rmse: 1.3124 - mse: 1.7666 - val_loss: 0.4592 - val_mae: 0.5099 - val_rmse: 1.1142 - val_mse: 1.2723
Epoch 2/140
394/394 [==============================] - ETA: 0s - loss: 0.5119 - mae: 0.6013 - rmse: 1.2443 - mse: 1.5859
LR: 9.999999747378752e-05 | Iter: 788 | Decay: 0.0

394/394 [==============================] - 21s 54ms/step - loss: 0.5119 - mae: 0.6013 - rmse: 1.2443 - mse: 1.5859 - val_loss: 0.4366 - val_mae: 0.4677 - val_rmse: 1.0452 - val_mse: 1.1243
Epoch 3/140
394/394 [==============================] - ETA: 0s - loss: 0.4996 - mae: 0.5501 - rmse: 1.1424 - mse: 1.3418
LR: 9.999999747378752e-05 | Iter: 1182 | Decay: 0.0

394/394 [==============================] - 22s 55ms/step - loss: 0.4996 - mae: 0.5501 - rmse: 1.1424 

Process Keras_worker_ForkPoolWorker-258:
Process Keras_worker_ForkPoolWorker-254:
Process Keras_worker_ForkPoolWorker-257:
Process Keras_worker_ForkPoolWorker-255:
Process Keras_worker_ForkPoolWorker-253:
Process Keras_worker_ForkPoolWorker-256:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self.

KeyboardInterrupt: 

In [ ]:
model.layers[-1].layer.alphabet

In [ ]:
%reload_ext tensorboard
#%tensorboard --logdir jag_train1/results_jag/logs/ --host 0.0.0.0 --port 6015
%tensorboard --logdir logs/ --host 0.0.0.0 --port 6014

In [ ]:
utils.save_obj(history.history,'history_padrao_ouro','./')
model.save('padrao_ouro.h5')
model.save_weights('padrao_ouro.h5')

In [ ]:
h = history.history

In [ ]:
utils.save_obj(h,'history_bilstm15_64cells_no_callbacks','.')

In [ ]:
teste = np.load('train_70_zmat/y_0001.npy')

In [ ]:
teste[:,1]

### Testing Angle Plot

## Analyses

In [ ]:
def rose_plot(ax, angles, bins=40, density=False, offset=0, lab_unit="degrees",
              start_zero=True, **param_dict):
    """
    Plot polar histogram of angles on ax. ax must have been created using
    subplot_kw=dict(projection='polar'). Angles are expected in radians.
    """
    # Wrap angles to [-pi, pi)
    angles = (angles + np.pi) % (2*np.pi) - np.pi

    # Set bins symetrically around zero
    if start_zero:
        # To have a bin edge at zero use an even number of bins
        if bins % 2:
            bins += 1
        bins = np.linspace(-np.pi, np.pi, num=bins+1)

    # Bin data and record counts
    count, bin = np.histogram(angles, bins=bins)

    # Compute width of each bin
    widths = np.diff(bin)

    # By default plot density (frequency potentially misleading)
    if density is None or density is True:
        # Area to assign each bin
        area = count / angles.size
        # Calculate corresponding bin radius
        radius = (area / np.pi)**.5
    else:
        radius = count

    # Plot data on ax
    ax.bar(bin[:-1], radius, zorder=1, align='edge', width=widths,
           edgecolor='C0', fill=False, linewidth=1)

    # Set the direction of the zero angle
    ax.set_theta_offset(offset)

    # Remove ylabels, they are mostly obstructive and not informative
    ax.set_yticks([])

    if lab_unit == "radians":
        label = ['$0$', r'$\pi/4$', r'$\pi/2$', r'$3\pi/4$',
                  r'$\pi$', r'$5\pi/4$', r'$3\pi/2$', r'$7\pi/4$']
        ax.set_xticklabels(label)

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]

SMALL = 10
MEDIUM = 14
BIG = 22

plt.rc('font', size=SMALL)          # controls default text sizes
plt.rc('axes', titlesize=SMALL)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL)    # legend fontsize
plt.rc('figure', titlesize=BIG)  # fontsize of the figure title

In [ ]:
def coords_to_pdb2(coords_sets):
    """ Write a chain to a PDB file """
    pdb_lines = ''
    counter = 1
    atom_types = [' N  ', ' CA ', ' C ']
    elements = ['N', 'C', 'C',]
    element_dict = dict(zip(atom_types, elements))
    # N CA C O SCOM
    # example = 'ATOM      1  N   HIS A 218      11.361  61.787  24.572  1.00 38.92           N  '
    pdb_lines = []
    for i in range(coords_sets.shape[0]): 
        #print(coords_sets[i].shape)
        for atom_type, X in zip(atom_types, coords_sets[i]):
            # N CA C O SCOM
            #  1 -  6        Record name   "ATOM  "
            line = 'ATOM  '
            # 7 - 11        Integer       serial       Atom  serial
            # number.
            line = line + str(counter).rjust(5) + ' '
            # 13 - 16        Atom          name         Atom name.
            line = line + atom_type
            # 17             Character     altLoc       Alternate
            # location indicator.
            line = line + ' '
            # 18 - 20        Residue name  resName      Residue name.
            line = line + 'ALA '
            # 22             Character     chainID      Chain
            # identifier.
            line = line + 'A'
            # 23 - 26        Integer       resSeq       Residue
            # sequence number.
            line = line + str(i + 1).rjust(4)
            # 27             AChar         iCode        Code for
            # insertion of residues.
            line = line + ' ' + '   '
            # 31 - 38        Real(8.3)     x            Orthogonal
            # coordinates for X in Angstroms.
            line = line + ('%.3f' % X[0]).rjust(8)
            # 39 - 46        Real(8.3)     y            Orthogonal
            # coordinates for Y in Angstroms.
            line = line + ('%.3f' % X[1]).rjust(8)
            # 47 - 54        Real(8.3)     z            Orthogonal
            # coordinates for Z in Angstroms.
            line = line + ('%.3f' % X[2]).rjust(8)
            # 55 - 60        Real(6.2)     occupancy    Occupancy.
            line = line + '  1.00'
            # 61 - 66        Real(6.2)     tempFactor   Temperature
            # factor.
            line = line + ' 12.00'
            # 77 - 78        LString(2)    element      Element symbol,
            # right-justified.
            line = line + '          ' + element_dict[atom_type]
            # 79 - 80        LString(2)    charge       Charge  on the
            # atom.
            pdb_lines.append(line)
            counter = counter + 1
    return pdb_lines

## Train Loss